In [1]:
import sys
sys.path.append("/home/watkinsz/Desktop/For_Zack/ML")
sys.path.append("/home/watkinsz/Desktop/For_Zack/ML/FeH")
from fitsParser.fitsParser import fitsParser
import pandas as pd
import numpy as np
import re
from modelValidator import psMetric, cut, shuffleDwarfs, conf_matrix, plotRes, plot, norm_inputs, monte_carlo, printTable, transformer_man
from sklearn.gaussian_process import GaussianProcessRegressor as gpr
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from function import save_data
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.neural_network import MLPClassifier as mlp
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Nadam


sdss = fitsParser(includefile = 'FeHdata.txt')
cut(sdss)
sdss.makeColours(N=10)
data = sdss.data.sample(frac=1)
l = len(data)
data['test_flag'] = [0]*int(0.8*l)+[1]*(l-int(0.8*l))
thresh = -2
classes = [0, thresh]

def getMags(colour):
    return colour[:colour.index('-')], colour[colour.index('-')+1:]

transformer = transformer_man(classes,
                             lambda x , c: x < thresh and c!=0 or x>=thresh and c==0,
                             lambda x: thresh*x.index(1))
data['label'] = transformer.transform(data['FeHadop'])
Encoder = LabelEncoder()
data['class'] = transformer.inverse_transform(data['label'])
data['class_label'] = Encoder.fit_transform(data['class'])

def monte(din, N=3):
    tous = pd.DataFrame(din)
    uncert = pd.DataFrame()
    for col in sdss.colours:
        c1 = 'd'+re.compile('.+(?=-)').findall(col)[0]
        c2 = 'd' + re.compile('(?<=-).+').findall(col)[0]
        uncert[col] = np.sqrt(tous[c1]**2+tous[c2]**2)

    uncert[map(lambda x : x[1:], sdss.output_uncert)] = tous[sdss.output_uncert]
    return monte_carlo(tous[list(sdss.colours+sdss.outputs)],uncert ,N=N)


dprelim = data[(data['class']==0)&(data['test_flag']==0)]
maxlen = len(dprelim)
print "Stars with [Fe/H]>%.1f: %d"%(thresh,maxlen)
train = dprelim
for c in classes[1:]:
    datac = pd.DataFrame(data[(data['class']==c)&(data['test_flag']==0)])
    l = len(datac)
    print 'Num stars in class %.1f: %d'%(c,l)
    if l!=0:
        datac = monte(datac, N = maxlen/len(datac)-1)
        train = train[datac.keys()]
        train = train.append(datac, sort=True).reset_index(drop=True)
colours = ['CaHK_0-g_fpsf', 'g_fpsf-i_fpsf', 'g_fpsf-r_fpsf', 'u_0-CaHK_0', 'u_0-g_fpsf']

Using TensorFlow backend.



Data saved

condition on dg_fpsf cut 22 inputs
condition on dr_fpsf cut 23 inputs
condition on di_fpsf cut 6 inputs
condition on dz_fpsf cut 2 inputs
condition on dy_fpsf cut 14 inputs
condition on dCaHK_0 cut 14 inputs
condition on du_0 cut 158 inputs
condition on dFeHadop cut 280 inputs
10.66% of inputs cut
Stars with [Fe/H]>-2.0: 26260
Num stars in class -2.0: 2488


In [2]:
train['class_label'] = Encoder.transform([0 if x>thresh else thresh for x in train['FeHadop']])
train['label'] = transformer.transform(train['FeHadop'])
params = {
    'hidden_layer_sizes' : (128,128,128,128,128,128),
    'alpha' : 1e-6,
    'batch_size' : 100,
    'tol' : 1e-6,
    'verbose' : True
}

#model = mlp(**params)
#model.fit(train[colours], train['class_label'])
#model.score(train[colours], train['class_label'])

In [3]:
def com_matrix(df, mets, classes, exp_class = 'class', pred_class = 'FeH_pred'):
    return [[len(data[data[pred_class]==c])*1.0/len(data) if len(data) != 0 else -1 for c in classes] for data in 
             [df[((mets[i]>df['FeHadop'])&(df['FeHadop']>mets[i+1]) if i!= len(mets)-1 else df['FeHadop']<mets[i-1])] for i in range(len(mets))]]

def pretty_print(matrix, rows, columns):
    display(pd.DataFrame(matrix, columns = columns, index = rows))
    
def purities(data, classes):
    return [[len(data[(data['FeH_pred']==c1)&(data['class']==c2)])*1.0/len(data[data['FeH_pred']==c1]) if len(data[data['FeH_pred']==c1])!= 0 else -1 for c2 in classes]
            for c1 in classes]


In [4]:
def printresults(data, mets, classes):
    print "purities:"
    pretty_print(purities(data, classes), rows =  classes, columns = classes)
    print "completnesses"
    pretty_print(conf_matrix(data,  classes, adop='class'), columns = classes, rows = classes)    
    pretty_print(com_matrix(data, mets, classes), rows = mets, columns = i)
i = [0,thresh]
mets = [0,-1,-2,-3,-4]
""""data['FeH_pred']= Encoder.inverse_transform(model.predict(data[colours]))





printresults(data, mets, classes)
print "\nTesting..."
printresults(data[data['test_flag']==1], mets, classes)
""" 


'"data[\'FeH_pred\']= Encoder.inverse_transform(model.predict(data[colours]))\n\n\n\n\n\nprintresults(data, mets, classes)\nprint "\nTesting..."\nprintresults(data[data[\'test_flag\']==1], mets, classes)\n'

In [5]:
def cont_frac(data, met, classes, adop = 'FeHadop'):
    return [[len(cont[(cont[adop]<met[i])&(cont[adop]>met[i+1]) if i != len(met)-1 else cont[adop]<met[i]])*
            1.0/len(cont) for cont in [data[(data['class']!=c)&(data['FeH_pred']==c)] for c in classes]]
            for i in range(len(met))]

#pretty_print(cont_frac(data, mets , classes), mets, classes)


In [6]:
youc = sdss.getAllObj()[1]
cut(youc)
youc.makeColours(N=10)
youc.data['class'] = [0 if x>= thresh else thresh for x in youc.data['FeHadop']]
#youc.data['FeH_pred'] = Encoder.inverse_transform(model.predict(youc.data[colours]))
#printresults(youc.data,[-2,-3,-4], classes)


Data saved

condition on dg_fpsf cut 1 inputs
condition on dr_fpsf cut 0 inputs
condition on di_fpsf cut 0 inputs
condition on dz_fpsf cut 0 inputs
condition on dy_fpsf cut 0 inputs
condition on dCaHK_0 cut 0 inputs
condition on du_0 cut 0 inputs
condition on dFeHadop cut 6 inputs
1.56% of inputs cut


In [7]:
def kerasmodel(inputsDim, outDim, Nlayers, Nnodes, **kwargs):
    model = Sequential()
    model.add(Dense(Nnodes, input_dim = inputsDim, activation = 'relu'))
    
    for i in range(Nlayers-1):
        model.add(Dense(Nnodes, activation = 'relu'))
    model.add(Dense(outDim, activation = 'softmax'))
    op = Nadam()
    model.compile(loss='binary_crossentropy', optimizer=op, metrics=['accuracy'])
    return model

kerasModel = kerasmodel(len(colours), 2, 6, 256)
params = {
    'x' : train[colours].values,
    'y' : np.array([x for x in train['label'].values]),
    'verbose' : 2,
    'batch_size': 800,
    'epochs' : 60,
    'validation_data' : 
    (data[data['test_flag']==1][colours].values, np.array([x for x in data[data['test_flag']==1]['label'].values]))
}

kerasModel.fit(**params)


Train on 51140 samples, validate on 7187 samples
Epoch 1/60
 - 5s - loss: 0.5550 - acc: 0.7388 - val_loss: 0.3723 - val_acc: 0.8545
Epoch 2/60
 - 2s - loss: 0.3972 - acc: 0.8252 - val_loss: 0.3406 - val_acc: 0.8408
Epoch 3/60
 - 2s - loss: 0.3643 - acc: 0.8373 - val_loss: 0.2956 - val_acc: 0.8613
Epoch 4/60
 - 2s - loss: 0.4846 - acc: 0.7764 - val_loss: 0.2618 - val_acc: 0.8731
Epoch 5/60
 - 2s - loss: 0.3882 - acc: 0.8308 - val_loss: 0.3482 - val_acc: 0.8311
Epoch 6/60
 - 2s - loss: 0.3530 - acc: 0.8441 - val_loss: 0.2339 - val_acc: 0.8870
Epoch 7/60
 - 2s - loss: 0.3278 - acc: 0.8546 - val_loss: 0.2705 - val_acc: 0.8623
Epoch 8/60
 - 2s - loss: 0.3291 - acc: 0.8552 - val_loss: 0.2227 - val_acc: 0.8958
Epoch 9/60
 - 2s - loss: 0.3292 - acc: 0.8557 - val_loss: 0.1599 - val_acc: 0.9327
Epoch 10/60
 - 2s - loss: 0.3120 - acc: 0.8632 - val_loss: 0.2914 - val_acc: 0.8554
Epoch 11/60
 - 2s - loss: 0.3068 - acc: 0.8660 - val_loss: 0.2940 - val_acc: 0.8568
Epoch 12/60
 - 2s - loss: 0.3046 - a

KeyboardInterrupt: 

In [15]:
kdata_sdss = pd.DataFrame(data)
mets = [0,-1.5,-2,-2.5,-3,-4]
kdata_sdss['FeH_pred'] = [0 if x[0] > x[1] else thresh for x in kerasModel.predict(kdata_sdss[colours])]
printresults(kdata_sdss, mets, classes)
kdata_youc = pd.DataFrame(youc.data)
kdata_youc['label'] = transformer.transform(kdata_youc['FeHadop'])
kdata_youc['FeH_pred'] = [0 if x[0] > x[1] else thresh for x in kerasModel.predict(kdata_youc[colours])]
print 'validation'
printresults(kdata_sdss[kdata_sdss['test_flag']==1],mets,classes)
print 'y. sample'

printresults(kdata_youc, mets, classes)

purities:


,0,-2
0,0.991616,0.008384
-2,0.612226,0.387774


completnesses


,0,-2
0,0.861440,0.138560
-2,0.076627,0.923373


,0,-2
0.0,0.946024,0.053976
-1.5,0.430209,0.569791
-2.0,0.096369,0.903631
-2.5,0.036923,0.963077
-3.0,0.000000,1.000000
-4.0,0.000000,1.000000


...

y. sample
purities:


,0,-2
0,0.022727,0.977273
-2,0.037037,0.962963


completnesses


,0,-2
0,0.040000,0.960000
-2,0.064468,0.935532


,0,-2
0.0,-1.000000,-1.000000
-1.5,-1.000000,-1.000000
-2.0,0.104478,0.895522
-2.5,0.004695,0.995305
-3.0,0.000000,1.000000
-4.0,0.000000,1.000000


In [ ]:
import joblib
def loadAll(filename):
    x  = [joblib.load(filename + end +".P") for end in ["", "Params", "Data"]]
    return x[0],x[1],x[2] #model, params, data

pretrans = lambda data : [[1 if False not in [x[i] > y or y == x[i] for y in x ] else 0 for i in range(len(x))] for x in data] 
testmodel, tmparams, tmdata = loadAll('PredlimClassVal1')
tmdata['bin_class'] = [-2 if x[1] > x[0] else 0 for x in kerasModel.predict(tmdata[colours])]
#tmdata['bin_class'] = Encoder.inverse_transform(model.predict(tmdata[colours]))
tmdata['FeHpred'] = [tmparams['classes'][x.index(1)] for x in pretrans(testmodel.predict(tmdata[tmparams['colours']]))]
data_sdss = pd.DataFrame(kdata_sdss)
data_sdss['bin_class'] = data_sdss['FeH_pred']
data_sdss['FeH_pred'] = [tmparams['classes'][x.index(1)] for x in pretrans(testmodel.predict(data_sdss[tmparams['colours']]))]

In [ ]:
data = pd.DataFrame(tmdata)
classes = tmparams['classes']
i = [-2,-2.5,-3]
pur, com = psMetric(data, inc = i)


data['FeH_pred'] = data['FeH_pred']*[0 if x==0 else 1 for x in data['bin_class']]
print 'all y.'
print printTable(i ,pur,com)
print pd.DataFrame(conf_matrix(data, classes), columns = classes, index = classes) , '\n'      
print 'test y'             
pur2, com2 = psMetric(data[(data['test_flag']==1)], inc = i)
print printTable(i,pur2,com2)
print pd.DataFrame(conf_matrix(data[(data['test_flag']==1)], classes), columns = classes, index = classes),'\n'  
print 'pristine'
pur3, com3 = psMetric(data, inc = i, pred = 'FeHround_prist')
print printTable(i,pur3,com3)
print pd.DataFrame(conf_matrix(data, classes, pred = 'FeHround_prist'), columns = classes, index = classes),'\n' 
print 'sdss'
pur4, com4 = psMetric(data_sdss, inc = i, pred = 'FeH_pred')
print printTable(i,pur4,com4)
print pd.DataFrame(conf_matrix(data, classes, pred = 'FeH_pred'), columns = classes, index = classes),'\n' 
print pretty_print(com_matrix(data_sdss, [0,-1]+i, classes), rows = [0,-1]+i, columns = classes)

In [ ]:
from functools import reduce
reduce(lambda y,x : y+(0 if x==[1,0] else 1), train['label'],0)
print len(sdss.data)